# Get manuscript 'timeline'

## Description
Given an OpenReview invitation, list the events (submissions, revisions, reviews, comments, etc.) associated with each submission, arranged chronologically.

## Notes
* I am not sure what the difference is between `cdate`, `tcdate` and other timestamps. I will check with OpenReview folks to confirm that I am using the correct field.
* The list of events does not include comment edits, including score changes. I do not think these are available, but I can double check with OpenReview folks on this too.
* I haven't done any of the work to print this into a nice csv or anything, but I think it should be straightforward. You can use `event.stringify(pretty=False)` if you would prefer raw timestamps instead of human readable.

In [4]:
import collections
from datetime import datetime
import openreview
import os

# Change these values according to your needs
INVITATION = 'ICLR.cc/2018/Conference/-/Blind_Submission'
LIMIT = 10 # Number of papers to download all revisions for

# A client is required for any OpenReview API actions
guest_client = openreview.Client(baseurl='https://api.openreview.net')

In [5]:
class EventType(object):
    SUBMISSION = "Submission"
    REVISION = "Revision"
    REVIEW = "Review"
    COMMENT = "Comment"
    METAREVIEW = "Metareview"
    
GENERIC_INITIATOR_AUTHORS = "Authors"

def get_initiator(note):
    return note.signatures[0].split("/")[-1]

def clean_timestamp(timestamp):
    return datetime.fromtimestamp(
        timestamp/1000).strftime('%Y-%m-%dT%H:%M:%S') # Something human-readable for the file name

class Event(object):
    def __init__(self, forum, event_type, initiator, identifier, timestamp, reply_to):
        self.forum = forum
        self.event_type = event_type
        self.initiator = initiator
        self.identifier = identifier
        self.timestamp = timestamp
        self.reply_to = reply_to
    
    def stringify(self, pretty=True):
        if pretty:
            timestamp = clean_timestamp(self.timestamp)
        else:
            timestamp = self.timestamp
        return "\t".join(str(i) for i in [
            timestamp, self.forum, self.event_type,
            self.initiator, self.identifier, self.reply_to])

def get_events_from_forum(forum_note):
    events = []
    for revision in guest_client.get_references(
            referent=forum_note.id, original=True):
        events.append(Event(
            forum_note.id, EventType.REVISION, GENERIC_INITIATOR_AUTHORS,
            revision.id, revision.tcdate, revision.replyto))
  
    this_forum_notes = guest_client.get_notes(forum=forum_note.id)
    for note in this_forum_notes:
        initiator = get_initiator(note)
        
        if 'TL;DR' in note.content: # Find a better test for this!
            event_type = EventType.SUBMISSION
        elif 'review' in note.content:
            event_type = EventType.REVIEW
        elif 'metareview' in note.content:
            event_type = EventType.METAREVIEW
        else:
            event_type = EventType.COMMENT
        events.append(Event(
            forum_note.id, event_type, initiator,
            note.id, note.tcdate, note.replyto))
    return list(sorted(events, key=lambda x:x.timestamp))

In [6]:
for i, forum_note in enumerate(openreview.tools.iterget_notes(
        guest_client, invitation=INVITATION)):
    
    print( "#" + forum_note.content["title"] + "\n")
    for event in get_events_from_forum(forum_note):
        print(event.stringify())
    print("\n")
    
    if i == LIMIT:
        break

#Parameter Space Noise for Exploration

2017-10-27T09:12:44	ByBAl2eAZ	Revision	Authors	SJB0xhgCZ	None
2017-10-27T09:12:45	ByBAl2eAZ	Submission	Conference	ByBAl2eAZ	None
2017-11-09T01:16:01	ByBAl2eAZ	Review	AnonReviewer3	By9jfdZkM	ByBAl2eAZ
2017-11-15T13:07:32	ByBAl2eAZ	Comment	Area_Chair	rkaPGW5JM	rk940apAW
2017-11-26T14:59:52	ByBAl2eAZ	Review	AnonReviewer2	r1gBpq_gG	ByBAl2eAZ
2017-11-27T06:52:12	ByBAl2eAZ	Review	AnonReviewer1	ryVd3dFgf	ByBAl2eAZ
2017-12-15T01:21:24	ByBAl2eAZ	Comment	Authors	SkaJ5JWMG	ryVd3dFgf
2017-12-15T01:22:26	ByBAl2eAZ	Comment	Authors	ByiX9J-zf	r1gBpq_gG
2017-12-15T01:23:09	ByBAl2eAZ	Comment	Authors	HJUU9kZzf	By9jfdZkM
2018-01-03T07:27:02	ByBAl2eAZ	Revision	Authors	ByyQ3S5mz	None
2018-01-03T07:28:35	ByBAl2eAZ	Comment	Authors	HJjO3r97f	ByBAl2eAZ
2018-01-05T20:50:54	ByBAl2eAZ	Comment	(anonymous)	HyUtjoTmG	ByBAl2eAZ
2018-01-08T05:59:03	ByBAl2eAZ	Comment	Authors	HJxWJ0eEG	HyUtjoTmG
2018-01-12T13:33:12	ByBAl2eAZ	Comment	AnonReviewer2	S1Zd1FUEM	ByiX9J-zf
2018-01-29T13:1

2017-10-26T14:44:26	rJXMpikCZ	Revision	Authors	HyMz6iy0-	None
2017-10-26T14:44:27	rJXMpikCZ	Submission	Conference	rJXMpikCZ	None
2017-10-27T14:00:42	rJXMpikCZ	Revision	Authors	H17UVgbR-	None
2017-10-27T14:07:35	rJXMpikCZ	Revision	Authors	HkxeUlb0-	None
2017-10-27T15:22:28	rJXMpikCZ	Revision	Authors	S1ndwZZAW	None
2017-10-27T16:36:08	rJXMpikCZ	Revision	Authors	Hyx6dMWRZ	None
2017-11-02T13:11:00	rJXMpikCZ	Comment	(anonymous)	B16obCO0W	rJXMpikCZ
2017-11-06T10:02:02	rJXMpikCZ	Comment	~Yedid_Hoshen1	ryGdFlCCb	rJXMpikCZ
2017-11-06T12:06:12	rJXMpikCZ	Comment	Authors	S12tIMACW	B16obCO0W
2017-11-06T12:18:24	rJXMpikCZ	Comment	~Michael_Bronstein1	r1KPYM0Cb	rJXMpikCZ
2017-11-08T08:52:23	rJXMpikCZ	Comment	~Thomas_N._Kipf1	HJk72Fekz	rJXMpikCZ
2017-11-08T12:08:06	rJXMpikCZ	Comment	Authors	H1Cechgkz	ryGdFlCCb
2017-11-08T12:24:42	rJXMpikCZ	Comment	Authors	HJMkC2xyz	r1KPYM0Cb
2017-11-08T19:41:11	rJXMpikCZ	Comment	Authors	BJg4NXZyz	HJk72Fekz
2017-11-13T10:13:50	rJXMpikCZ	Comment	(anonymous)	HyL2UVDJG	rJX